In [49]:
import os
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import re

### Setup

In [50]:
MONTH = 'September 2019'
FOLDER = 'U:/Data and Analytics/Projects/Data/MOCJ ' + MONTH

In [51]:
CRIM_NAMES = ["Case#", "Open Date", "Closed Date", "Court", "Latest Event",
              "Intake Type", "TC C/L", "Top Charge", "Dispo"]
POST_DISPO_NAMES = ["Case#", "Open Date", "Closed Date", "Court", "Latest Event",
                    "Intake Type", "TC C/L", "Top Charge", "Dispo", "Case Type"]

MOCJ_CASE_TYPES = CategoricalDtype(categories=["Homicide",
                                               "Violent Felony",
                                               "Non-Violent Felony",
                                               "Misdemeanor",
                                               "Violation & Other"],
                                   ordered=True)
MOCJ_INTAKE_TYPES = CategoricalDtype(categories=["Primary Case", "Conflict Case", "Post Arraignment"],
                                     ordered=True)
MOCJ_OUTFLOW_TYPES = CategoricalDtype(categories=["Post-Arraignment Shift Transfer",
                                                  "Post-Arraignment Disposition",
                                                  "Arraignment Shift Disposition"],
                                      ordered=True)

Functions for classifying top count and intake type, to translate from PDCMS categories to MOCJ categories.

In [52]:
def classify_intake_type(intake_type):
    """Classifies intake type into MOCJ-requested categories, which are:
    New Cases - Primary Case at Arraignment Shift
    New Cases - Conflict Case at Arraignment Shift
    New Cases - Post Arraignment Shift Other Intake
    """
    intake_type = str(intake_type).strip().lower()
    if (intake_type in ["arraignment", "row"]):
        return ("Primary Case")
    elif (intake_type in ["conflict row", "conflict arraignment"]):
        return ("Conflict Case")
    elif (intake_type.startswith("transfer")) | (intake_type == "assignment"):
        return ("Post Arraignment")
    else:
        return ("DANGER")

In [53]:
def classify_tc(tc):
    """Classifies top charge class and levels into MOCJ-requested categories.
    """
    tc = str(tc) #coerce TC type to string
    if tc == "H":
        return ("Homicide")
    elif ("FL" in tc) or ("FV" in tc):
        return ("Violent Felony")
    elif (tc.endswith("F") or tc == "F"):
        return ("Non-Violent Felony")
    elif (tc.endswith("M") or tc == "M"):
        return ("Misdemeanor")
    else:
        return ("Violation & Other")

In [54]:
def categorize_disposition(row):
    if (row["Open Date"] == row["Closed Date"]):
        return "Arraignment Shift Disposition"
    elif (row["Dispo"] in ["RPC", "RLAS", "R18B", "RNDS", "ADBX", "ADBR", "ADQU", "ADRI"]):
        return "Post-Arraignment Shift Transfer"
    else:
        return "Post-Arraignment Shift Disposition"

## Criminal Cases
Criminal cases opened and closed in the month

In [55]:
crim_cases_opened = pd.read_csv("{}/crim_cases_opened.csv".format(FOLDER),
                                names=CRIM_NAMES,
                                header=0,
                               index_col=False)
crim_cases_closed = pd.read_csv("{}/crim_cases_closed.csv".format(FOLDER),
                                names=CRIM_NAMES,
                                header=0,
                               index_col=False)


crim_cases = pd.concat([crim_cases_opened, crim_cases_closed])
print(crim_cases.shape)

crim_cases = crim_cases.drop_duplicates().reset_index(drop=True)
print(crim_cases.shape)

(1398, 9)
(1115, 9)


In [56]:
#crim_cases["Case#"].value_counts()
#crim_cases[crim_cases["Case#"] == "*"]
    #*insert any case # that comes up in the value counts here

In [57]:
crim_cases_pending_1st = pd.read_csv("{}/crim_cases_pending_1st.csv".format(FOLDER),
                                     names=CRIM_NAMES, header=0, index_col=False)
crim_cases_pending_end = pd.read_csv("{}/crim_cases_pending_last.csv".format(FOLDER),
                                      names=CRIM_NAMES, header=0, index_col=False)

crim_cases = pd.concat([crim_cases,
                        crim_cases_pending_1st,
                        crim_cases_pending_end])
print(crim_cases.shape)

(5961, 9)


In [58]:
crim_cases = crim_cases.dropna(how="all")
print(crim_cases.shape)

crim_cases = crim_cases.drop_duplicates(subset=["Case#", "Open Date", "Closed Date", "Intake Type", "Dispo"],
                                        keep='last').reset_index(drop = True)
print(crim_cases.shape)

# Checking whether a known duplicate has been de-duplicated
crim_cases[crim_cases["Case#"] == '21-709207']

(5961, 9)
(3194, 9)


,Case#,Open Date,Closed Date,Court,Latest Event,Intake Type,TC C/L,Top Charge,Dispo
999,21-709207,3/9/2018,NaN,51,10/24/19 - NCD,Arraignment,FL,1252501 - MURDER 2 (AFL),NaN


In [59]:
crim_cases["Open Date"] = pd.to_datetime(crim_cases["Open Date"])
crim_cases["Closed Date"] = pd.to_datetime(crim_cases["Closed Date"])

### Cleaning cases
We don't want non-criminal IDV, VOP, or VOCD cases in these counts, so we're getting rid of cases where the top charge is either VISITATION or FAMILY OFFENSE.

Some top charges have more than one class and level, so we consolidate those.

In [60]:
# Getting rid of IDV/Visitation cases
crim_cases = crim_cases[~crim_cases['Top Charge'].str.contains("VISITATION", na=False)]
crim_cases = crim_cases[~crim_cases['Top Charge'].str.contains('FAMILY OFFENSE', na=False)]
crim_cases = crim_cases.loc[~((crim_cases["Court"] == "IDV")
                              & (crim_cases["Intake Type"] == "Assignment"))]
crim_cases = crim_cases.loc[~((crim_cases["Court"] == "IDV")
                              & (crim_cases["Intake Type"].str.startswith("Trans")))]

In [61]:
# Re-code homicide cases to a separate Homicide class, but don't include attempted homicides
# Finds top charges that have "MUR" but don't have "ATT"
crim_cases.loc[crim_cases["Top Charge"].str.contains('MUR', na=False) \
                & ~(crim_cases["Top Charge"].str.contains("ATT", na=False)), "TC C/L"] = "H"
crim_cases.loc[crim_cases["Top Charge"].str.contains("- MUR", na=False), "TC C/L"] = "H"

In [62]:
crim_cases[crim_cases["TC C/L"] == "H"]

,Case#,Open Date,Closed Date,Court,Latest Event,Intake Type,TC C/L,Top Charge,Dispo
817,21-118538,2016-05-07,NaT,61,10/10/19 - NCD,Arraignment,H,1252501 - MURDER 2 (AFL),NaN
999,21-709207,2018-03-09,NaT,51,10/24/19 - NCD,Arraignment,H,1252501 - MURDER 2 (AFL),NaN
1640,21-721516,2019-03-26,NaT,71,10/21/19 - NCD,Assignment,H,1252501 - MURDER 2 (AFL),NaN
2325,21-724754,2019-07-19,NaT,62,11/21/19 - NCD,Arraignment,H,1252501 - MURDER 2 (AFL),NaN
2381,21-724912,2019-07-27,NaT,32,10/30/19 - NCD / Sw Assigned,Arraignment,H,1252501 - MURDER 2 (AFL),NaN
2831,21-725966,2019-09-06,NaT,71,12/09/19 - NCD,Arraignment,H,1252501 - MURDER 2 (AFL),NaN
3100,21-726444,2019-08-28,NaT,71,12/09/19 - NCD,Assignment,H,1252503 - MURDER 2 (AFL),NaN


In [63]:
# Re-code TC C/L for extradition cases
crim_cases.loc[(crim_cases["Top Charge"].str.contains("FUG", na=False)), 'TC C/L'] = "M"

In [64]:
# AC Misdemeanors don't get counted correctly
crim_cases.loc[(crim_cases["Top Charge"].str.contains("AC - M", na=False)), 'TC C/L'] = "M"

We also check to see if any VOCDs or VOPs have snuck in.

In [65]:
# Check for any VOCD or VOP cases, then fix in PDCMS until there are no more cases returned
crim_cases[(crim_cases['Top Charge'].str.contains("VIO", na=False))
           & (crim_cases['Top Charge'].str.contains('COND', na=False))]
vocd_case = crim_cases[(crim_cases['Top Charge'].str.contains("VIO", na=False))
           & (crim_cases['Top Charge'].str.contains('COND', na=False))]
crim_cases = crim_cases[~((crim_cases['Top Charge'].str.contains("VIO", na=False))
           & (crim_cases['Top Charge'].str.contains('COND', na=False)))]

In [66]:
# Check for any VOCD or VOP cases, then fix in PDCMS until there are no more cases returned
crim_cases[(crim_cases['Top Charge'].str.contains("VOP", na=False))]

,Case#,Open Date,Closed Date,Court,Latest Event,Intake Type,TC C/L,Top Charge,Dispo


### Determining Start and End

Because this is now a combined dataset, we need to find out whether the cases were opened and/or closed in the month.

In [67]:
crim_cases[crim_cases["Open Date"] < pd.to_datetime('2019-09-01')]["Open Date"].describe()

count                    2458
unique                    552
top       2019-08-08 00:00:00
freq                       45
first     2004-02-06 00:00:00
last      2019-08-31 00:00:00
Name: Open Date, dtype: object

In [68]:
def check_in_month(dt):
    """Check if a date is within the same month and year
    """
    
    month = pd.datetime.today().month -1
    year = pd.datetime.today().year
    return True if (dt.year == year) and (dt.month == month) else False

crim_cases["opened_in_month"] = crim_cases["Open Date"].apply(lambda x: check_in_month(x))
crim_cases["closed_in_month"] = crim_cases["Closed Date"].apply(lambda x: check_in_month(x))

In [69]:
# Because the dataset is only cases that either opened or closed in the month
# we know that any case not opened in this month will have been pending as of the 1st,
# and we know that any case not closed in this month will have been pending as of the
# last of the month.

crim_cases["pending_first"] = crim_cases["Open Date"].apply(lambda x: True if x < pd.to_datetime('2019-09-01') else False)

In [70]:
crim_cases["pending_last"] = crim_cases["Closed Date"].apply(
                            lambda x: True if ((x > pd.to_datetime('2019-09-30')) | pd.isnull(x)) else False)

In [71]:
crim_cases[crim_cases.pending_first == True][["Open Date", "Closed Date"]].describe()

,Open Date,Closed Date
count,2458,457
unique,552,28
top,2019-08-08 00:00:00,2019-09-16 00:00:00
freq,45,33
first,2004-02-06 00:00:00,2019-09-03 00:00:00
last,2019-08-31 00:00:00,2019-10-17 00:00:00


In [72]:
crim_cases[crim_cases.pending_last == True][["Closed Date", "Open Date"]].describe()

,Closed Date,Open Date
count,56,2452
unique,5,549
top,2019-10-03 00:00:00,2019-09-12 00:00:00
freq,24,45
first,2019-10-01 00:00:00,2004-02-06 00:00:00
last,2019-10-17 00:00:00,2019-09-30 00:00:00


### Classifying Top Charges
Now we translate the top charge type to MOCJ categories and check that everything worked OK.

In [73]:
#crim_cases["TC C/L"].value_counts()
#crim_cases[crim_cases["TC C/L"] == "/I"] #(or any one you want to cherck out)

In [74]:
crim_cases["TC_Type"] = crim_cases["TC C/L"].apply(lambda x: classify_tc(x))
crim_cases.groupby(["TC_Type", "TC C/L"]).size()

TC_Type             TC C/L
Homicide            H            7
Misdemeanor         AM         121
                    BM           9
                    M         1400
                    UM          20
Non-Violent Felony  AF           2
                    BF           4
                    CF           1
                    DF           7
                    EF           2
                    F          887
Violation & Other   /I          10
                    0          190
                    0V           2
Violent Felony      BFV          1
                    DFV          4
                    FL          17
                    FV         465
dtype: int64

In [75]:

crim_cases["TC_Type"] = crim_cases.TC_Type.astype(MOCJ_CASE_TYPES)

### Classifying Intake Type

Now we translate the intake type to MOCJ categories and check that everything is OK.

We are assuming that everything with a PDCMS intake type of "Assignment" and a court other than IDV is probably actually a transfer.

In [76]:
#crim_cases_closed["intake type"] = crim_cases_closed["TC C/L"].apply(lambda x: classify_tc(x))
crim_cases["intake type"] = crim_cases["Intake Type"].apply(lambda x: classify_intake_type(x))

crim_cases["intake type"].value_counts()

Primary Case        2323
Conflict Case        628
Post Arraignment     200
Name: intake type, dtype: int64

In [77]:
crim_cases[crim_cases["intake type"] == "DANGER"]

,Case#,Open Date,Closed Date,Court,Latest Event,Intake Type,TC C/L,Top Charge,Dispo,opened_in_month,closed_in_month,pending_first,pending_last,TC_Type,intake type


In [78]:
# Make intake type into a Pandas categorical type, so pivot tables are sorted
crim_cases["intake type"] = crim_cases["intake type"].astype(MOCJ_INTAKE_TYPES)

### Classifying Outflow Type

Translate the disposition to MOCJ categories.

In [79]:
crim_cases["MOCJ_Dispo"] = crim_cases.apply(lambda x: categorize_disposition(x), axis=1)
crim_cases.groupby(["MOCJ_Dispo", "Dispo"]).size()

MOCJ_Dispo                          Dispo
Arraignment Shift Disposition       ACD       71
                                    ADBR       9
                                    ADBX       4
                                    ADQU      17
                                    ADRI       5
                                    DISM      21
                                    FACD       1
                                    MACD       1
                                    PLEA       2
                                    PLSE      95
                                    R18B       1
                                    RES        2
                                    RLAS       1
                                    RNDS       2
                                    RPC        1
Post-Arraignment Shift Disposition  730        7
                                    ABAT       2
                                    ACD       30
                                    ACQ        2
                           

In [80]:
crim_cases.groupby(["MOCJ_Dispo"]).size()

MOCJ_Dispo
Arraignment Shift Disposition          233
Post-Arraignment Shift Disposition    2808
Post-Arraignment Shift Transfer        110
dtype: int64

### Final Aggregate Counts

In [81]:
crim_cases[crim_cases.opened_in_month == True].pivot_table(values=["Case#"],
                                                                index=["TC_Type", "intake type"],
                                                                aggfunc=np.count_nonzero)

Case#
TC_Type            intake type            
Homicide           Primary Case        1.0
Violent Felony     Primary Case       39.0
                   Conflict Case      17.0
                   Post Arraignment    4.0
Non-Violent Felony Primary Case       83.0
                   Conflict Case      20.0
                   Post Arraignment    8.0
Misdemeanor        Primary Case      279.0
                   Conflict Case      68.0
                   Post Arraignment   23.0
Violation & Other  Primary Case      135.0
                   Conflict Case      10.0
                   Post Arraignment    6.0

In [82]:
crim_cases[crim_cases.closed_in_month == True].pivot_table(values=["Case#"],
                                                           index=["TC_Type", "MOCJ_Dispo"],
                                                           aggfunc=np.count_nonzero)

Case#
TC_Type            MOCJ_Dispo                               
Violent Felony     Arraignment Shift Disposition         1.0
                   Post-Arraignment Shift Disposition   26.0
                   Post-Arraignment Shift Transfer      12.0
Non-Violent Felony Arraignment Shift Disposition         2.0
                   Post-Arraignment Shift Disposition   80.0
                   Post-Arraignment Shift Transfer      21.0
Misdemeanor        Arraignment Shift Disposition       107.0
                   Post-Arraignment Shift Disposition  257.0
                   Post-Arraignment Shift Transfer      46.0
Violation & Other  Arraignment Shift Disposition       123.0
                   Post-Arraignment Shift Disposition   12.0
                   Post-Arraignment Shift Transfer      12.0

In [83]:
crim_cases[crim_cases.pending_first == True].pivot_table(values=["Case#"], index=["TC_Type"],
                                                         aggfunc=np.count_nonzero)

,Case#
TC_Type,
Homicide,6
Violent Felony,427
Non-Violent Felony,792
Misdemeanor,1180
Violation & Other,53


In [84]:
crim_cases[crim_cases.pending_last == True].pivot_table(values=["Case#"], index=["TC_Type"],
                                                        aggfunc=np.count_nonzero)

,Case#
TC_Type,
Homicide,7
Violent Felony,448
Non-Violent Felony,800
Misdemeanor,1140
Violation & Other,57


In [85]:
crim_cases["case_age"] = (crim_cases["Closed Date"] - crim_cases["Open Date"]).dt.days
crim_cases.case_age.describe()

count     755.000000
mean      108.369536
std       243.225305
min         0.000000
25%         0.000000
50%        41.000000
75%       109.000000
max      3552.000000
Name: case_age, dtype: float64

In [86]:
crim_cases[crim_cases.closed_in_month == True].groupby('TC_Type').agg({'case_age': 'describe'})

case_age                                                   \
                      count        mean         std  min   25%    50%    75%   
TC_Type                                                                        
Homicide                0.0         NaN         NaN  NaN   NaN    NaN    NaN   
Violent Felony         39.0  177.846154  184.125892  0.0  45.0  113.0  247.0   
Non-Violent Felony    103.0  264.388350  440.029817  0.0  37.5  132.0  302.0   
Misdemeanor           410.0   93.729268  193.576144  0.0   0.0   48.5  100.5   
Violation & Other     147.0   22.006803  155.237974  0.0   0.0    0.0    0.0   

                            
                       max  
TC_Type                     
Homicide               NaN  
Violent Felony       717.0  
Non-Violent Felony  3552.0  
Misdemeanor         3031.0  
Violation & Other   1805.0

In [87]:
crim_cases[crim_cases.case_age > 200][["Case#", "Court", "Latest Event", "Top Charge", "Dispo", "case_age"]]

,Case#,Court,Latest Event,Top Charge,Dispo,case_age
293,21-125229,B,"09/20/2019, RW, ROW",THEFT OF SERV,DISM,1100.0
294,21-141565,41,"09/20/2019, NCD, DISPOSED",AGGR UNLIC OP 1,RPC,755.0
295,21-144790,AR3,"09/11/2019, RW, ROW",CRIM TRESPASS 3,DISM,671.0
296,21-145087,B,"09/20/2019, RW, SENTENCED",CRIM MISCHIEF 4,PLSE,674.0
297,21-145745,AR1,"09/16/2019, RW, ROW",CPSP 5,ACD,655.0
298,21-714756,D,"09/19/2019, NCD, SENTENCED",CRIM CONTEMPT 2,PLSE,421.0
299,21-714757,D,"09/19/2019, NCD, DISPOSED",OTHER (UNK),DISS,421.0
300,21-717448,VTL,"09/16/2019, RW, SENTENCED",OP VEH LIC SUSP,PLSE,319.0
301,21-719755,A,"09/16/2019, NCD, SEALED",CPCS 3,DISS,240.0
302,21-719793,VTL,"09/16/2019, RW, ROW",UNLIC OPERATION,DISM,243.0


## Post-Disposition Cases

In [88]:
pd_cases = pd.concat([pd.read_csv("{}/post_dispo_opened.csv".format(FOLDER),
                                  names=POST_DISPO_NAMES,
                                  header=0),
                      pd.read_csv("{}/post_dispo_closed.csv".format(FOLDER),
                                  names=POST_DISPO_NAMES,
                                  header=0),
                      pd.read_csv("{}/post_dispo_pending_1st.csv".format(FOLDER),
                                  names=POST_DISPO_NAMES,
                                  header=0),
                      pd.read_csv("{}/post_dispo_pending_last.csv".format(FOLDER),
                                  names=POST_DISPO_NAMES,
                                  header=0),
                      vocd_case], sort = True)

pd_cases = pd_cases.drop_duplicates(subset=["Case#", "Open Date", "Closed Date"],
                                    keep='last').reset_index(drop = True)
pd_cases = pd_cases.dropna(subset = ["Case Type","Case#","Court","Intake Type", "Latest Event", "Open Date", "TC C/L", "Top Charge"])

pd_cases["Case Type"] = pd_cases["Case Type"].str.upper()

In [89]:
pd_cases

,Case Type,Case#,Closed Date,Court,Dispo,Intake Type,Latest Event,Open Date,TC C/L,Top Charge
0,VOP,21-103721,NaN,32,NaN,ROW,"10/23/2019, VOP, NEW ATTY ASSIGNED",9/18/2019,EF,VIOL PROB SENT VIOL F
1,VOP,21-113712,NaN,61,NaN,ROW,"11/21/2019, NCD, NEW ATTY ASSIGNED",9/5/2019,EF,VIOL PROB SENT VIOL F
2,VOP,21-136218,NaN,D,NaN,ROW,"10/28/2019, NCD, No Result",9/3/2019,U,VIOL PROB SENT VIOL
3,VOP,21-137899,NaN,93,NaN,ROW,"10/15/2019, NCD, No Result",9/11/2019,UV,FAILURE TO PAY FINE
4,VOCD,21-139333,NaN,E,NaN,ROW,"11/18/2019, NCD, No Result",9/16/2019,9,VIOLATE CONDITION OF S
5,VOCD,21-723306,NaN,41,NaN,ROW,"09/13/2019, NCD, NEW ATTY ASSIGNED",9/5/2019,9,VIOLATE CONDITION OF S
6,VOP,21-91935V,NaN,71,NaN,ROW,"10/07/2019, NCD, No Result",9/16/2019,EF,VIOL PROB SENT VIOL F
7,VOCD,21-132794,9/20/2019,B,PLSE,ROW,"09/20/2019, RW, SENTENCED",9/20/2019,0,VIOL COND DISC (DC)
8,VOP,21-709739,9/6/2019,B,R18B,ROW,"09/06/2019, VOP, DISPOSED",7/25/2019,AM,VIOL PROB SENT VIOL M
9,VOCD,21-711618,9/5/2019,AR2,PLSE,ROW,"09/05/2019, RW, SENTENCED",9/5/2019,9,VIOLATE CONDITION OF S


Create dummy variables for whether the case was opened and/or closed in the month.

In [90]:
pd_cases["Open Date"] = pd.to_datetime(pd_cases["Open Date"])
pd_cases["Closed Date"] = pd.to_datetime(pd_cases["Closed Date"])
pd_cases["opened_in_month"] = pd_cases["Open Date"].apply(lambda x: check_in_month(x))
pd_cases["closed_in_month"] = pd_cases["Closed Date"].apply(lambda x: check_in_month(x))

Create a dummy variable for pending on the 1st of the month or the last.

In [91]:
pd_cases["pending_first"] = pd_cases["Open Date"].apply(lambda x: True if x < pd.to_datetime('2019-09-01') else False)

In [92]:
pd_cases["pending_last"] = pd_cases["Closed Date"].apply(
                            lambda x: True if ((x > pd.to_datetime('2019-09-30')) | pd.isnull(x)) else False)

In [93]:
pd_cases[pd_cases.pending_first == True].groupby("Case Type").size()

Case Type
VOCD    10
VOP     15
dtype: int64

In [94]:
pd_cases[pd_cases.opened_in_month == True].groupby("Case Type").size()

Case Type
VOCD    17
VOP     10
dtype: int64

In [95]:
pd_cases[pd_cases.closed_in_month == True].groupby("Case Type").size()

Case Type
VOCD    17
VOP      3
dtype: int64

In [96]:
pd_cases[pd_cases.pending_last == True].groupby("Case Type").size()

Case Type
VOCD    10
VOP     22
dtype: int64